<a href="https://colab.research.google.com/github/Luis7123/amortizacion/blob/main/payments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar librerias.

In [94]:
# Importacion de librerias
import pandas as pd

In [ ]:
from pandas._libs.tslibs.period import period_asfreq
# @title Funcion Calcular Cuota

# Excepcions /excepciones

class ExcesiveInterestException( Exception ):
    pass
class montoInexistente( Exception ):
    pass
class montoNegativo( Exception ):
    pass
class plazoNegativo( Exception ):
    pass
class InteresNegativo( Exception ):
    pass

# import libraries / importar librerias

import pandas as pd
def calcularCuota (amount, interest_rate, periods ):
    '''
    It calculates the monthly payment for a purchase amount with a interest rate in a number of periods

    Parameters
        ----------

        amount : float
            Purchase amount / Valor de la compra
        interest : float
            Monthly interest rate for purchase. Must be zero or positive less than
            MAX_INTEREST_RATE / Tasa maxima de interes, valor positivo menor que MAX_INTEREST_RATE
        periods : int
            Number of monthly payments / Numero de cuotas a diferir la compra

        Returns
        -------
        payment : float
            Monthly payment calculated. Not rounded / Pago mensual calculado. El resultado no esta redondeado

    '''

    if interest_rate == 0 :
        '''
        if the interest equals to cero it give just the division between amount and payment time.
        '''
        return amount/periods

    if interest_rate < 0:
        '''
         Raises
        ------
        ExcesiveInterestException
            When interest rate is above the valu defined in  MAX_INTEREST_RATE

        The interest rate cannot  be a negative number.
        '''
        raise InteresNegativo( "The interest rate must be positive")

    if interest_rate*12 > 100:
        '''
        Interest rate must be between maximum 0 and 99 per year
        '''
        raise ExcesiveInterestException( "The annual interest rate cannot exceed 100 percent ")

    if periods == 1:
        return 0

    if amount <= 0:
        if amount== 0 :
            raise montoInexistente( "debe existir un monto")
        else:
            raise montoNegativo( "monto negativo")

    if periods <= 0:
        if periods== 0 :
            return 0
        else:
            raise plazoNegativo( "The periods it's a number of time, must be positive")
    else:
        # Calculo de la cuota luego de filtros
        i= interest_rate/100
        payment=(amount * i) / (1 - (1 + i) ** (-periods))
        return (payment)


In [ ]:
# @title DataFrame amortization
class lowDeposit( Exception ):
    """
    Custom exception for low deposit
    //
    Excepcion personalizada para indicar que el deposito es bajo

    """
    pass
class paymentLate( Exception ):
    '''
    Custom exception for paying in a month too late
    //
    Excepcion para cuando se paga el abono en un mes muy adelantado.
    '''

    pass

def dataframe_amortization(amount,interest_rate,period,deposit=0,month_deposit=0):
    '''
    DataFrame of every month , the payment, the periods, the amortization and the debt

    '''
    amount = amount
    interest = interest_rate
    period = period
    amortization=0
    interest_payment=0

    payment=calcularCuota(amount, interest, period)
    payment_Cambio=calcularCuota (amount, interest, period)
    listadf=[[payment,interest,amortization,amount]]

    # Comprobar que el abono sea superior
    if deposit !=0:
      if deposit < payment:
        raise lowDeposit( " The deposit is too low ")

    # Ciclo de la amortizacion.
    for i in range(period):
        '''
        loop that creates a list of the variables and calculate its amortization
        taking into consideration
        '''
        if month_deposit-1 == i : # Conditional that deposit the payment in the corresponding month
          if deposit > amount:
            raise paymentLate(" The amount paid is too high ")

          payment = deposit
          interes_cuota=amount*interest/100
          amortization=payment-amount*interest/100
          amount=amount-payment+interes_cuota
        else:
          payment = payment_Cambio
          interes_cuota=amount*interest/100
          amortization=payment-amount*interest/100
          amount=amount-payment+interes_cuota

        #Filtros de negativos
        if amount < payment:
          listadf.append([payment,interes_cuota,amortization,round(amount,3)])

          payment =amount + amount*interest/100
          interes_cuota=amount*interest/100
          amortization=payment-amount*interest/100
          amount=amount-payment+interes_cuota

        listadf.append([payment,interes_cuota,amortization,round(amount,3)])

        if amount <= 1*10**-3:
          break
    return pd.DataFrame(listadf, columns=["Payment","Interest","amortization","balance"])

In [96]:
# @title Interfaz

'''
on this interface you are going to write and put whatever you want to know
For dataFrame of the amortizations, the value of the payment that you have to give

Even if you want to deposit an amount in a specific month.
'''
amortization= True # @param {type:"boolean"}
Payment_cal = True # @param {type:"boolean"}

Amount = 850000 # @param {type:"number"}
Interest = 3.4 # @param {type:"number"}
Period = 10 # @param {type:"number"}
Want_to_deposit = True # @param {type:"boolean"}
if Want_to_deposit ==True:
  Deposit = 190000 # @param {type:"number"}
  Which_month = 5 # @param {type:"number"}


if amortization == True:
  if Want_to_deposit==True:
    display(dataframe_amortization(Amount,Interest,Period,Deposit , Which_month))
  else:
    display(dataframe_amortization(Amount,Interest,Period))
if Payment_cal == True:
  print("\n La cuota mensual será : ",calcularCuota (Amount, Interest, Period))


,Payment,Interest,amortization,balance
0,101690.672541,3.400000,0.000000,850000.000
1,101690.672541,28900.000000,72790.672541,777209.327
2,101690.672541,26425.117134,75265.555408,701943.772
3,101690.672541,23866.088250,77824.584292,624119.188
4,101690.672541,21220.052384,80470.620157,543648.568
5,190000.000000,18484.051298,171515.948702,372132.619
6,101690.672541,12652.509043,89038.163499,283094.455
7,101690.672541,9625.211484,92065.461058,191028.994
8,101690.672541,6494.985808,95195.686734,95833.308
9,99091.640069,3258.332459,95833.307611,0.000



 La cuota mensual será :  101690.67254128284


In [ ]:
dataframe_amortization(Amount,Interest,Period,Deposit , Which_month)

[52377.49863983651, 28900.0, 23477.498639836507, 826522.501]